# Add/Remove SSH Keys

This guide outlines the process of granting access to virtual machines (VMs) within your slice to other users. When collaborating with others, it's common to encounter situations where you need to enable them to access the VMs running in your slice. Access to these VMs on FABRIC is managed through SSH keys. This notebook offers methods to incorporate the SSH keys of your collaborators into the VMs within your slice, ensuring they have the necessary access to your VMs.


## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

### **Use a Collaborator's Slice**  

Access an existing slice where you have been granted permission and add SSH keys.

In [ ]:
my_existing_slice = True
slice_name = 'MySlice-lan'

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  

**NOTE**: Skip the following cells if you choose to use and existing slice as indicated above.

In [ ]:
if not my_existing_slice:
    slice_name = 'SSH-Keys'
    site = fablib.get_random_site()
    print(f"Sites: {site}")
    
    node1_name='Node1'
    node2_name='Node2'
    
    network_name='net1'
    
    from ipaddress import IPv4Network
    
    subnet = IPv4Network("192.168.1.0/24")
    
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    net1 = slice.add_l2network(name=network_name, subnet=subnet)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site)
    iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
    iface1.set_mode('auto')
    net1.add_interface(iface1)
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site)
    iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
    iface2.set_mode('auto')
    net1.add_interface(iface2)
    
    
    #Submit Slice Request
    slice.submit();

### **Get Slice Information**  

You can retrieve information about a slice using the following command:  

```python
slice = fablib.get_slice(slice_name)
```

In [ ]:
slice = fablib.get_slice(slice_name)

## Adding SSH Keys 

### **Option 1: Grant a User Access to Your Slice by Adding Their SSH Keys**

Allow a user to access nodes in your slice by adding their SSH keys. The recommended approach is to use the **sliver_key_name** from the [FABRIC Portal](https://portal.fabric-testbed.net/experiments#sshKeys) for seamless authentication.  

#### Adding Your Own SSH Key  
If you are adding your own key, specify only the `sliver_key_name`:  
```python
node.add_public_key(sliver_key_name=sliver_key_name)
```

#### Adding Another User's SSH Key  
If you are adding another user's SSH key, you must also provide their email address:  
```python
node.add_public_key(sliver_key_name=sliver_key_name, email=email)
```

In [ ]:
sliver_key_name = "REPLACE_WITH_COLLABORATOR_SLIVER_KEY_NAME"
email = "REPLACE_WITH_COLLABORATOR_EMAIL"

sliver_key_name = "slice_key"
email = "ktharej@ncsu.edu"

In [ ]:
for node in slice.get_nodes():
    node.add_public_key(sliver_key_name=sliver_key_name, email=email)

### **Option 2: Add an SSH Key Using a Public Key String**

If you have the public key string that needs to be added manually, you can pass it directly to the node. This method allows you to add any valid SSH public key without relying on the **sliver_key_name** from the FABRIC portal.  

```python
node.add_public_key(sliver_public_key=public_key_string)
```

This approach is useful when:  
- The SSH key is not registered in the FABRIC portal.  
- You have received a collaborator’s public key in text format.  
- You need to add a temporary or externally generated key.  

Make sure the provided `public_key_string` is correctly formatted and corresponds to the private key the user will use for SSH access.

In [ ]:
public_key_string = "REPLACE_WITH_PUBLIC_KEY_STRING"

# Comment it when using the public key string, set to None to ensure the next cell is skipped
public_key_string = None

In [ ]:
if public_key_string:
    for node in slice.get_nodes():
        node.add_public_key(public_key_string=public_key_string)

### **Verify That the User's SSH Key Has Been Added**

After adding the SSH key, confirm that it has been successfully integrated into the node by checking the authorized keys:  

```python
node.execute("cat ~/.ssh/authorized_keys")
```

If the key was added correctly, you should see the corresponding public key entry in the output.

At this point, ask the user to verify if they can access your VMs by attempting to SSH into the node using their credentials:  
```
ssh -i <Path to Private Sliver Key> -F <Path to SSH Config> <username>@<MgmtIP>
```
If the SSH key was added correctly, they should be able to log in without a password.

In [ ]:
for node in slice.get_nodes():
    node.execute("cat ~/.ssh/authorized_keys")
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

## **Removing SSH Keys**
If you need to revoke access for a user, you can remove their SSH key from the node. There are two ways to do this, depending on how the key was added.


## **Option 1: Remove an SSH Key Using `sliver_key_name`**  

If the SSH key was added using the **sliver_key_name** from the [FABRIC Portal](https://portal.fabric-testbed.net/experiments#sshKeys), you can remove it using the same identifier.  

### Removing Your Own SSH Key  
If you are removing your own key, specify only the `sliver_key_name`:  
```python
node.remove_public_key(sliver_key_name=sliver_key_name)
```

### Removing Another User’s SSH Key  
If you are removing another user's key, provide their **email address** along with the `sliver_key_name`:  
```python
node.remove_public_key(sliver_key_name=sliver_key_name, email=email)
```

This ensures that only the specified user’s key is removed while keeping others intact.

In [ ]:
sliver_key_name = "REPLACE_WITH_COLLABORATOR_SLIVER_KEY_NAME"
email = "REPLACE_WITH_COLLABORATOR_EMAIL"

sliver_key_name = "slice_key"
email = "ktharej@ncsu.edu"

In [ ]:
for node in slice.get_nodes():
    # Add Collaborator's Portal Public Key to the Node by passing the sliver key name and email
    node.remove_public_key(sliver_key_name=sliver_key_name, email=email) 

### **Option 2: Remove an SSH Key Using a Public Key String**  

If the SSH key was added manually as a string, you can remove it by passing the same public key string.  

#### Removing a Public Key String from the Node  
```python
node.remove_public_key(sliver_public_key=public_key_string)
```

This method is useful when:  
- The key was not registered with a **sliver_key_name** in the FABRIC portal.  
- You manually added the key and now need to remove it using the exact string.  
- You want to ensure a specific key is removed without affecting others.  

In [ ]:
public_key_string = "REPLACE_WITH_PUBLIC_KEY_STRING"

# Comment it when using the public key string, set to None to ensure the next cell is skipped
public_key_string = None

In [ ]:
if public_key_string:
    for node in slice.get_nodes():
        
        # Add Public Key to the Node by passing the public key string
        node.remove_public_key(sliver_public_key=public_key_string)

### **Verifying Key Removal**  

After removing a key, confirm that it is no longer present in the node’s `authorized_keys` file:  

```python
node.execute("cat ~/.ssh/authorized_keys")
```

If the key was removed successfully, it should no longer appear in the output.

In [ ]:
for node in slice.get_nodes():
    node.execute("cat ~/.ssh/authorized_keys")
    print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
if not my_existing_slice:
    slice = fablib.get_slice(slice_name)
    slice.delete()